# 05 Tooling | 01 Semantic Kernel | 04 - Process

## Step 1: Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.

This notebooks highlights how Semantic Kernel simplifies the process of storing and retrieving vector data. 


In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.48.0"
#r "nuget: Microsoft.SemanticKernel.Process.Core, 1.48.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.LocalRuntime, 1.48.0-alpha"
#r "nuget: DotNetEnv, 3.1.1"

using System.ClientModel; 
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Process;
using DotNetEnv;

string configurationFile = "../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";


Console.WriteLine($"Configuration loaded...");

Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.48.0 Microsoft.SemanticKernel.Process.Core, 1.48.0-alpha Microsoft.SemanticKernel.Process.LocalRuntime, 1.48.0-alpha

Configuration loaded...


## Step 2: Define Process Step Functionality

Semantic Kernel can orchestrate execution of specific process steps. Those steps are defined here. Within the sample a simplified sport event result prediction is provided. Therefore three process steps are defined:

- `QueryDataSources()`: Simulates the retrieval process of historical information. The historical data will be created using a model call.
- `CheckHistoricalInformation()`: A simulated check if the retrieved historical information is valid. 
- `PredictResult()`: A prediction of a future score


In [2]:
#pragma warning disable SKEXP0080

public class RetrieveHistoricalInformation: KernelProcessStep
{
    [KernelFunction]
    public async Task<string> QueryDataSources(Kernel kernel, string teamName)
    {
        //Using a llm to generate fictional data - Other plugins could be used to query real data
        Console.WriteLine($"\nRetrieve Historical Information");
        Console.WriteLine($"\t Collecting historical information for: {teamName}");

        string prompt = @"
            You create fictional results for the last 10 matches of the {{$teamName}}.
            You response with one line per match.
            You just provide the score.
            You don't provide the team name as part of the result of a match.
            The scores should be between 15 and 50.
            The scores should be separated by a colon.

            Create fictional results!
        ";

        KernelArguments kernelArguments = new KernelArguments() { 
            ["teamName"] = teamName
        };

        FunctionResult functionResult = await kernel.InvokePromptAsync(prompt, kernelArguments);
        string response = functionResult.GetValue<string>() ?? "";
        Console.WriteLine($"\t Historical information retrieved...");
        return response;
    }
}


Console.WriteLine($"'RetriveHistoricalInformation' defined...");

'RetriveHistoricalInformation' defined...


In [3]:
#pragma warning disable SKEXP0080

public class CheckHistoricalInformation: KernelProcessStep
{
    [KernelFunction]
    public async Task PerformCheck(Kernel kernel, KernelProcessStepContext context, string historicalData)
    {
        //Simulating a check of the historical data
        Console.WriteLine($"\nCheck historical information");
        Console.WriteLine($"\tCheck historical information: \n\t {historicalData.Replace("\n", "\n\t")}");

        //Generate random number between 0 and 2 to simulate check of historical data
        Random random = new Random(DateTime.Now.Millisecond);
        long randomNumber = random.Next(0, 3);
        
        if (randomNumber > 1 )
        {
            Console.WriteLine($"\t Check failed: Recreating historical data");
            await context.EmitEventAsync(
                eventId: "RecreateHistoricalData", 
                data: historicalData
            );
        }
        else
        {
            Console.WriteLine($"\t Check passed: Using historical data");
            await context.EmitEventAsync(
                eventId: "UseHistoricalData", 
                data: historicalData
            );
        }
    }
}

Console.WriteLine($"'CheckHistoricalInformation' defined...");


'CheckHistoricalInformation' defined...


In [4]:
#pragma warning disable SKEXP0080

public class PredictResult: KernelProcessStep
{
    [KernelFunction]
    public async Task PredictResultCalculation(Kernel kernel, KernelProcessStepContext context, string historicalData)
    {
        //Using a llm to predict a result
        Console.WriteLine($"\nPredicting Result");
        Console.WriteLine($"\tStart Predicting");

        string prompt = @"
            You predict the result of a sport match.
            You get 10 previous results.
            You just respond with one fictional result.
            You just provide the result.
            You don't provide any further information.

            Predict the result based on this historical information: {{$historicalData}} !
        ";

        KernelArguments kernelArguments = new KernelArguments() { 
            ["historicalData"] = historicalData
        };

        FunctionResult functionResult = await kernel.InvokePromptAsync(prompt, kernelArguments);
        string response = functionResult.GetValue<string>() ?? "";

        Console.WriteLine($"\tPredicted Result: {response}");

    }
}

Console.WriteLine($"'PredictResult' defined...");

'PredictResult' defined...


## Step 3: Process Definition

Within the code cell process steps and how they can communicate with each other are defined.

In [5]:
#pragma warning disable SKEXP0080

ProcessBuilder processBuilder = new("PredictMatchWinner");

// Add the steps
ProcessStepBuilder retrieveHistoricalInformation = processBuilder.AddStepFromType<RetrieveHistoricalInformation>();
ProcessStepBuilder checkHistoricalInformation = processBuilder.AddStepFromType<CheckHistoricalInformation>();
ProcessStepBuilder predictResult = processBuilder.AddStepFromType<PredictResult>();


processBuilder
    .OnInputEvent("PredictMatchResult")
    .SendEventTo(new (retrieveHistoricalInformation));

retrieveHistoricalInformation
    .OnFunctionResult()
    .SendEventTo(new (checkHistoricalInformation));

checkHistoricalInformation
    .OnEvent("RecreateHistoricalData")
    .SendEventTo(new (retrieveHistoricalInformation));

checkHistoricalInformation
    .OnEvent("UseHistoricalData")
    .SendEventTo(new (predictResult));

KernelProcess process = processBuilder.Build();

Console.WriteLine($"Process defined...");


Process defined...


## Step 4: Kernel Definition and Process Start

The defined process will be executed.

In [6]:
#pragma warning disable SKEXP0080

Kernel kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        apiKey: oAiApiKey, 
        endpoint: oAiEndpoint, 
        deploymentName: chatCompletionDeploymentName
    )
    .Build();

Console.WriteLine($"Kernel created...");

KernelProcessContext kernelProcessContext = await process.StartAsync(
    kernel, 
    new KernelProcessEvent { 
        Id = "PredictMatchResult", 
        Data = "Munich Flying Dolphins" 
    }
);
Console.WriteLine($"\nProcess finished...");

Kernel created...

Retrieve Historical Information
	 Historical information retrieved...

Check historical information
	Check historical information: 
	 35:29  
	41:33  
	28:24  
	47:36  
	30:27  
	38:34  
	45:22  
	33:30  
	24:28  
	49:31
	 Check passed: Using historical data

Predicting Result
	Start Predicting
	Predicted Result: 38:30

Process finished...
